In [1]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import os
import base64
from PIL import Image, ImageOps
from io import BytesIO
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
img_h = 32
img_w = 32
channels = 1

In [3]:
def prep_input_img(img):
#     img = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
    if img is not None:
        img=~img
        ret,thresh=cv2.threshold(img,127,255,cv2.THRESH_BINARY)
        ctrs,h=cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        cnt=sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
        w=int(img_w)
        h=int(img_h)
        train_data=[]
        rects=[]
        for c in cnt :
            x,y,w,h= cv2.boundingRect(c)
            rect=[x,y,w,h]
            rects.append(rect)
        bool_rect=[]
        for r in rects:
            l=[]
            for rec in rects:
                flag=0
                if rec!=r:
                    if r[0]<(rec[0]+rec[2]+10) and rec[0]<(r[0]+r[2]+10) and r[1]<(rec[1]+rec[3]+10) and rec[1]<(r[1]+r[3]+10):
                        flag=1
                    l.append(flag)
                if rec==r:
                    l.append(0)
            bool_rect.append(l)
        dump_rect=[]
        for i in range(0,len(cnt)):
            for j in range(0,len(cnt)):
                if bool_rect[i][j]==1:
                    area1=rects[i][2]*rects[i][3]
                    area2=rects[j][2]*rects[j][3]
                    if(area1==min(area1,area2)):
                        dump_rect.append(rects[i])
        final_rect=[i for i in rects if i not in dump_rect]
        for r in final_rect:
            x=r[0]
            y=r[1]
            w=r[2]
            h=r[3]
            im_crop =thresh[y:y+h+10,x:x+w+10]
            im_resize = cv2.resize(im_crop,(img_h,img_w))
            im_resize=np.reshape(im_resize,(img_h,img_w,channels))
            train_data.append(im_resize)
    return train_data

In [4]:
def predict(img_arr):
    s=''
    for i in range(len(img_arr)):
        img_arr[i]=np.array(img_arr[i])
        img_arr[i]=img_arr[i].reshape(1,img_h,img_w,channels)
        result=model.predict_classes(img_arr[i])
        print(int(result))
        if(result[0]==12):
            s=s+'-'
        if(result[0]==11):
            s=s+'+'
        if(result[0]==10):
            s=s+'*'
        if(result[0]==0):
            s=s+'0'
        if(result[0]==1):
            s=s+'1'
        if(result[0]==2):
            s=s+'2'
        if(result[0]==3):
            s=s+'3'
        if(result[0]==4):
            s=s+'4'
        if(result[0]==5):
            s=s+'5'
        if(result[0]==6):
            s=s+'6'
        if(result[0]==7):
            s=s+'7'
        if(result[0]==8):
            s=s+'8'
        if(result[0]==9):
            s=s+'9'
    return s

In [5]:
model = load_model('model/model32x32-1639256834')